## Data Pre-processing

> In this notebook we have pre-process data we have collected in Data collection. 
> The result is written to file for data Visualization

### import libraries

In [68]:
from pymongo import MongoClient
import re
import string
import dateutil.parser

### Connect to DB

In [2]:
client = MongoClient()
db = client['craigslist']
freebies_coll = db['freebies']

### Process Document from DB

In [55]:
def remove_new_line(_str):
    return _str.replace('\n', " ") 

In [56]:
def replace_multiple_space(_str):
    return re.sub(' +',' ',_str)

In [57]:
def remove_html_tags(_str):
    return re.sub('<[^>]*>', '', _str)

In [66]:
def remove_punctuation(_str):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in _str if ch not in exclude)
    #return _str.translate(None, string.punctuation)

In [64]:
def process_text(_str):
    new_str = _str.lower()
    new_str = remove_new_line(new_str)
    new_str = replace_multiple_space(new_str)
    new_str = remove_html_tags(new_str)
    new_str = remove_punctuation(new_str)
    return new_str

In [74]:
def process_date(_str):
    return dateutil.parser.parse(_str).strftime("%Y-%m-%d %H:%M:%S")

In [81]:
def process_doc(doc):
    new_doc = {}
    new_doc['id'] = str(doc['_id'])
    new_doc['title'] = process_text(doc['title'])
    new_doc['desc'] =  process_text(doc['desc'])
    new_doc['posteddate'] = process_date(doc['posteddate'])
    if 'location' in doc:
        new_doc['latitude'] = str(doc['location']['latitude'])
        new_doc['longitude'] = str(doc['location']['longitude'])
        new_doc['accuracy'] = str(doc['location']['accuracy'])
    return new_doc

In [98]:
def format_string(doc):
    s = '%s|%s|%s|%s|'%(doc['id'],doc['title'],doc['desc'],doc['posteddate'])
    if 'latitude' in doc:
        l = '%s|%s|%s'%(doc['latitude'],doc['longitude'],doc['accuracy'])
    else:
        l = '||'
    return s+l

### process document

In [101]:
lines = []
cursor = freebies_coll.find()
for doc in cursor: 
    lines.append(format_string(process_doc(doc)))

### write to file

In [110]:
with open('dat.csv','w') as f:
    for line in lines:
        f.write(line.encode('ascii', 'ignore')+'\n')